In [1]:
import numpy
import numpy.random as rng
import theano
import theano.tensor as T
import theano.printing as pr
import theano.d3printing as d3p
from theano.d3printing.formatting import GraphFormatter
from theano import ProfileMode

from IPython.display import SVG

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
%load_ext autoreload
%autoreload 2

# Model 

In [3]:
nsamples = 10000
ninput = 100
nhidden = 50
noutput = 1

def init_params(m, n):
    W = np.random.normal(0, 1, (m, n)).astype(theano.config.floatX)
    return theano.shared(W, borrow=True)

W1 = init_params(ninput, nhidden)
W2 = init_params(nhidden, noutput)
X = T.dmatrix('X')

H = T.nnet.sigmoid(T.dot(X, W1))
Z = T.nnet.sigmoid(T.dot(H, W2))
zm = T.mean(Z)

predict_profiled = theano.function([X], [Z, zm], profile=True)
data = rng.rand(nsamples, ninput)
dZ, dzm = predict_profiled(data)

predict_unprofiled = theano.function([X], [Z, zm], profile=False)

# Timining information via debugpring 

In [4]:
pr.debugprint(predict_profiled)


Timing Info
-----------
--> <time> <% time> - <total time> <% total time>'

<time>         computation time for this node
<% time>       fraction of total computation time for this node
<total time>   time for this node + total times for this node's ancestors
<% total time> total time for this node over total computation time

N.B.:
* Times include the node time and the function overhead.
* <total time> and <% total time> may over-count computation times
  if inputs to a node share a common ancestor and should be viewed as a
  loose upper bound. Their intended use is to help rule out potential nodes
  to remove when optimizing a graph because their <total time> is very low.

Elemwise{ScalarSigmoid}[(0, 0)] [@A] ''   9 --> 1.16e-04s  0.9% 1.21e-02s 95.4%
 |Dot22 [@B] ''   6 --> 9.38e-04s  7.4% 1.20e-02s 94.5%
   |Elemwise{ScalarSigmoid}[(0, 0)] [@C] ''   4 --> 4.59e-03s 36.1% 1.11e-02s 87.1%
   | |Dot22 [@D] ''   2 --> 6.48e-03s 51.0% 6.48e-03s 51.0%
   |   |X [@E]
   |   |<TensorType(

# Profiled graphed 

In [7]:
d3p.d3print(predict_profiled, 'predict_profiled.html')

The output file is available at predict_profiled.html


[open](./predict_profiled.html) 

In [ ]:
pr.pydot

In [13]:
pr.pydotprint(predict_profiled, 'predict_profiled', format='svg')

The output file is available at predict_profiled.svg


![predict_profiled.svg](predict_profiled.svg)

# Unprofiled graph

In [14]:
d3p.d3print(predict_unprofiled, 'predict_unprofiled.html')

The output file is available at predict_unprofiled.html


[open](./predict_unprofiled.html) 